# Implementação 1: Toy Example

Esse notebook apresenta a primeira implementação do Toy Example proposto.
Sua proposta segue os seguintes objetivos:
* A implementação não seguirá padrões sugeridos nos campos das dimensões da IAR.
* Esse notebook servirá de checagem para análise da eficiência e corretude da checklist proposta pelo NIAR.

## Processamento dos dados

Os dados são de origem do Datasus, contendo informações das internações nos períodos de 2022 à novembro de 2025 (dados mais recentes quando feito esse Toy Example).

Devido ao tamanho dos dados brutos, não se encontram na página do Github. No entanto, a tabela final (após todo o pré processamento) se encontra em "Tabela_lag_final.zip". Apenas essa tabela é necessária para a execução do notebook. Para tal, descomprima o arquivo zip e coloque o arquivo "Tabela_lag_final.csv" (o único arquivo csv) dentro do diretório "SIH_Dados/".

O download dos dados foi feito por conexão ftp com o servidor disponibilizado pelo datasus: ftp.datasus.gov.br. Especificamente, os dados se encontram no diretório "/dissemin/publicos/SIHSUS/200801_/Dados/". Foram usados todos os arquivos que começam com "RD" e contém "22", "23", "24", "25", no quinto e sexto caractére (quando escrito esse notebook, exclui-se os dados de dezembro de 2025 e os dados de novembro de 2025 dos estados "AC" e "RR", pois não foram divulgados ainda).

A fim de evitar refazer grandes cálculos toda vez que esse notebook é executado, o processamento dos dados - filtragem de colunas de interesse e cálculo de novas colunas - foi feito previamente. No diretório SIH_Dados, encontram-se os scripts usados para cada etapa do processamento:
* dbc_dbf.py e dbf_csv.py: Transformam os arquivos de dados do formato dbc para dbf e dbf para csv, respectivamente.
* filtrar.py: Lê os arquivos csv, filtram as colunas (e o "target" de interesse, DIAG_PRIN = "J...") e salvam em novos csv.
* tabelacao_agregado.py: Lê os novos arquivos csv, e calcula os valores agregados dos dados, agrupando-os por mes/ano e hospital. A tabela resultante é salva em "Tabela_Agregada.csv".
* tabelacao_lag_final.py: Lê a tabelacao do passo anterior e calcula as variaveis "lag", usadas no modelo para predicao. A tabela, com todos os atributos que serão usados para o modelo final, é salva em um último csv: "Tabela_lag_Final.csv".

O intuito de separar cada processo em seu próprio script é devido a robustez contra a possibilidade de erros ou mudanças, permitindo que alterações sejam feitas sem a necessidade de recomeçar o processo do zero. Além disso, cada script demora vários minutos, alguns até passando de uma hora, para terminar. Juntar todos os scripts em um resultaria em um programa que terminaria após várias horas.

### Etapa 1: Inicialização
Import's usados, e leitura dos dados 

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn as sk
import statsmodels.discrete.discrete_model as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error


df = pd.read_csv("./SIH_Dados/Tabela_lag_Final.csv")

### Etapa 2: Limpagem e separação dos dados
Retirada de linhas com dados nulos e separação dos dados de treinamento, validação e teste.

In [2]:
df.dropna(inplace=True)


X_Train = df[(df.year >= 2022) & (df.year <= 2024)]
X_Train = X_Train[(X_Train.year < 2024) | (X_Train.month <= 6)].reset_index(drop=True)
Y_Train = X_Train["J_count"]
X_Train.drop(columns="J_count", inplace=True)

X_Valid = df[(df.year == 2024) & (df.month >= 7)].reset_index(drop=True)
Y_Valid = X_Valid["J_count"]
X_Valid.drop(columns="J_count", inplace=True)

X_Test = df[df.year == 2025].reset_index(drop=True)
Y_Test = X_Test["J_count"]
X_Test.drop(columns="J_count", inplace=True)


### Etapa 3: Modelo Base
Testes usando modelo de Regressão Binomial Negativa, implementada pelo statsmodels

In [3]:

# Retirada de colunas com valor único
stm_train = X_Train.drop(columns=[c for c in X_Train.columns if X_Train[c].nunique() == 1])
stm_valid = X_Valid.drop(columns=[c for c in X_Valid.columns if X_Valid[c].nunique() == 1])
stm_test = X_Test.drop(columns=[c for c in X_Test.columns if X_Test[c].nunique() == 1])
# Evita matriz singular

model = sm.NegativeBinomial(Y_Train, stm_train)
res = model.fit()
print(res.summary())

val_resul = model.predict(params=res.params, exog=stm_valid)
test_resul = model.predict(params=res.params, exog=stm_test)

print(val_resul)
print(test_resul)

/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1074: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1567: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + exposure + offset)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1568: RuntimeWarning: invalid value encountered in multiply
  return -np.dot(L*X.T, X)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1568: RuntimeWarning: invalid value encountered in dot
  return -np.dot(L*X.T, X)
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/discrete/discrete_model.py:1478: RuntimeWarning: overflow encountered in exp
  L =

         Current function value: nan
         Iterations: 0
         Function evaluations: 2
         Gradient evaluations: 2


/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/luis_edu/Documents/Prog/NIAR/Toy_Example/venv/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                     NegativeBinomial Regression Results                      
Dep. Variable:                J_count   No. Observations:                56250
Model:               NegativeBinomial   Df Residuals:                    56175
Method:                           MLE   Df Model:                           74
Date:                Tue, 27 Jan 2026   Pseudo R-squ.:                     nan
Time:                        10:33:05   Log-Likelihood:                    nan
converged:                      False   LL-Null:                   -2.5300e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
year                                       nan        nan        nan        nan         nan         nan
month                                      nan        na

### Etapa 4: Modelo LightGBM

Testes usando modelo com objetivo Poisson, implementada pelo lightgbm

In [4]:
seed = 778

lgb_train = lgb.Dataset(X_Train, Y_Train)
lgb_valid = lgb.Dataset(X_Valid, Y_Valid)


model = lgb.train(params={"objective": "poisson"}, train_set=lgb_train, valid_sets=lgb_valid, callbacks=[lgb.early_stopping(stopping_rounds=5)])

res_pred = model.predict(X_Test, num_iteration=model.best_iteration)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18308
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 81
[LightGBM] [Info] Start training from score 3.504180
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's poisson: -91.0448


### Etapa 5: Métricas

Comparação dos resultados de cada modelo

In [5]:
df_resultados = X_Test[["month", "CNES", "hospital_porte"]]
df_resultados["year"] = 2025
df_resultados = df_resultados[["year", "month", "CNES", "hospital_porte"]]
df_resultados["Valor Real"] = Y_Test
df_resultados["Base resultados"] = test_resul
df_resultados["LightGBM resultados"] = res_pred

# TEMPORARIO ==============
test_resul = np.nan_to_num(test_resul, nan=0, copy=True)
df_resultados["Base resultados"] = test_resul
print(test_resul)

base_mae = mean_absolute_error(y_true=Y_Test, y_pred=test_resul)
lgb_mae = mean_absolute_error(y_true=Y_Test, y_pred=res_pred)


base_rmse = mean_squared_error(y_true=Y_Test, y_pred=test_resul)
lgb_rmse = mean_squared_error(y_true=Y_Test, y_pred=res_pred)

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

base_smape = smape(y_true=Y_Test, y_pred=test_resul)
lgb_smape = smape(y_true=Y_Test, y_pred=res_pred)

print(df_resultados)
print()
print("------------------------------ RESULTADOS GLOBAIS ------------------------------")
print("\t\t\t|\t\t\t\t|")
print("\tMetrica\t\t|\t\tBase\t\t|\tLightGBM")
print("------------------------+-------------------------------+-----------------------")
print("MAE\t\t\t|", base_mae, "\t\t|", lgb_mae)
print("\t\t\t|\t\t\t\t|")
print("RMSE\t\t\t|", base_rmse, "\t\t|", lgb_rmse)
print("\t\t\t|\t\t\t\t|")
print("SMAPE\t\t\t|", base_smape, "\t\t\t|", lgb_smape)
print("\t\t\t|\t\t\t\t|")



[0. 0. 0. ... 0. 0. 0.]
       year  month     CNES  hospital_porte  Valor Real  Base resultados  \
0      2025      1  2000385               3          21              0.0   
1      2025      1    10456               3         133              0.0   
2      2025      1    10499               3         190              0.0   
3      2025      1    10502               3          97              0.0   
4      2025      1  6876617               3          64              0.0   
...     ...    ...      ...             ...         ...              ...   
33972  2025     11  2006197               1          10              0.0   
33973  2025     11  2010631               2          12              0.0   
33974  2025     11  5616298               3          39              0.0   
33975  2025     11  2010151               2          29              0.0   
33976  2025     11  2722399               1          30              0.0   

       LightGBM resultados  
0                30.896444  
1    

In [6]:

for i in range (0, 4):
    print("")
    df_porte = df_resultados[df_resultados.hospital_porte == i]
    y_t = df_porte["Valor Real"]
    y_base = df_porte["Base resultados"]
    y_lgb = df_porte["LightGBM resultados"]
    
    base_mae = mean_absolute_error(y_true=y_t, y_pred=y_base)
    lgb_mae = mean_absolute_error(y_true=y_t, y_pred=y_lgb)

    base_rmse = mean_squared_error(y_true=y_t, y_pred=y_base)
    lgb_rmse = mean_squared_error(y_true=y_t, y_pred=y_lgb)

    base_smape = smape(y_true=y_t, y_pred=y_base)
    lgb_smape = smape(y_true=y_t, y_pred=y_lgb)

    t = "pequeno"
    if (i == 1):
        t = "medio"
    if (i == 2):
        t = "grande"
    if (i == 3):
        t = "extra"
    
    print("--------------------------- RESULTADOS: Porte", t, "---------------------------")
    print("\t\t\t|\t\t\t\t|")
    print("\tMetrica\t\t|\t\tBase\t\t|\tLightGBM")
    print("------------------------+-------------------------------+-----------------------")
    print("MAE\t\t\t|", base_mae, "\t\t|", lgb_mae)
    print("\t\t\t|\t\t\t\t|")
    print("RMSE\t\t\t|", base_rmse, "\t\t|", lgb_rmse)
    print("\t\t\t|\t\t\t\t|")
    print("SMAPE\t\t\t|", base_smape, "\t\t\t|", lgb_smape)
    print("\t\t\t|\t\t\t\t|")
    print()


--------------------------- RESULTADOS: Porte pequeno ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 8.882534299367967 		| 5.136817538831749
			|				|
RMSE			| 125.44797286881455 		| 46.46381783335173
			|				|
SMAPE			| 200.0 			| 59.35268921926631
			|				|


--------------------------- RESULTADOS: Porte medio ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 16.81654759161097 		| 8.034987421740105
			|				|
RMSE			| 413.3724360451717 		| 126.59496794192522
			|				|
SMAPE			| 200.0 			| 49.065643169462554
			|				|


--------------------------- RESULTADOS: Porte grande ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 29.446459412780655 		| 12.249351026216452
			|